In [1]:
import pandas as pd
import numpy as np
import os
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from env import host, user, password
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.cluster import KMeans
from env import get_db_url

import seaborn as sns
import matplotlib.pyplot as plt
import wrangle as w
import explore as exp
from wrangle import clean_zillow
# modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
# Vis imports to make things look pretty 
import graphviz
from sklearn.tree import export_graphviz
import matplotlib.style as style 
style.available
style.use('seaborn-talk') #sets the size of the charts
style.use('seaborn-colorblind') #sets color of charts
from wrangle import wrangle_zillow


# ignore warnings
import warnings
warnings.filterwarnings("ignore")

<img src="stampede.tif"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 20px;" />

## <span style="color:maroon;"> In the Event of Clustering use the Elbow Method !!! </span>

In [2]:
df = w.zillow17()

In [3]:
df = w.clean_zillow(df)

In [4]:
df.head().T

parcelid,14297519,17052889,14186244,12177905,10887214
bathrooms,3.5,1.0,2.0,3.0,3.0
bedrooms,4,2,3,4,3
property_quality,7,7,7,8,8
sqft,3100.0,1465.0,1243.0,2376.0,1312.0
fips,6059,6111,6059,6037,6037
latitude,33634931.0,34449266.0,33886168.0,34245180.0,34185120.0
longitude,-117869207.0,-119281531.0,-117823170.0,-118240722.0,-118414640.0
lot_sqft,4506.0,12647.0,8432.0,13038.0,278581.0
rawcensustractandblock,60590630.072012,61110010.023006,60590218.022012,60373001.001006,60371236.012
regionidcity,53571.0,13091.0,21412.0,396551.0,12447.0


In [8]:
df.shape

(71818, 23)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71818 entries, 14297519 to 12826780
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   bathrooms               71818 non-null  float64 
 1   bedrooms                71818 non-null  int64   
 2   property_quality        71818 non-null  int64   
 3   sqft                    71818 non-null  float64 
 4   fips                    71818 non-null  int64   
 5   latitude                71818 non-null  float64 
 6   longitude               71818 non-null  float64 
 7   lot_sqft                71818 non-null  float64 
 8   rawcensustractandblock  71818 non-null  float64 
 9   regionidcity            71818 non-null  float64 
 10  zip_code                71818 non-null  int64   
 11  roomcnt                 71818 non-null  int64   
 12  unitcnt                 71818 non-null  int64   
 13  yearbuilt               71818 non-null  int64   
 14  structure_va

In [23]:
from sklearn.model_selection import train_test_split

# Create test dataset

train_validate, test = train_test_split(df, 
                                        test_size=.2, 
                                        random_state=123) 
                                        
train, validate = train_test_split(train_validate, 
                                   test_size=.3, 
                                   random_state=123) 
                                   

# Take a look at your split datasets

print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

train -> (40217, 23)
validate -> (17237, 23)
test -> (14364, 23)


In [24]:
train.shape, validate.shape, test.shape

((40217, 23), (17237, 23), (14364, 23))

In [25]:
train.columns.to_list()

['bathrooms',
 'bedrooms',
 'property_quality',
 'sqft',
 'fips',
 'latitude',
 'longitude',
 'lot_sqft',
 'rawcensustractandblock',
 'regionidcity',
 'zip_code',
 'roomcnt',
 'unitcnt',
 'yearbuilt',
 'structure_value',
 'home_value',
 'land_value',
 'taxamount',
 'logerror',
 'transactiondate',
 'county',
 'home_age',
 'logerror_quartiles']

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
explore.head().T

In [ ]:
def train_validate_test_split(df, target, seed):
    '''
    spilts our data  into train, validate, test
    '''
    # Train, Validate, and test
    train_and_validate, test = train_test_split(
        df, test_size=0.2, random_state=seed)
    train, validate = train_test_split(
        train_and_validate,
        test_size=0.3,
        random_state=seed)
    # Split with X and y
    X_train = train.drop(columns=[target])
    y_train = train[target]
    X_validate = validate.drop(columns=[target])
    y_validate = validate[target]
    X_test = test.drop(columns=[target])
    y_test = test[target]
    return train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test  

In [ ]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = train_validate_test_split(explore, 'logerror', 123)

In [ ]:
print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

In [ ]:
train.head().T

#columns_to_scale=['bedroomcnt', 'calculatedfinishedsquarefeet', 'fullbathcnt', 'lotsizesquarefeet', 'roomcnt', 'unitcnt', 
#'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'taxamount', 'longitude', 'latitude', 'age', 'taxrate', 'structure_dollar_per_sqft', 
#'land_dollar_per_sqft', 'bed_bath_ratio']

In [ ]:
def Standard_Scaler(X_train, X_validate, X_test):
    """
    Takes in X_train, X_validate and X_test dfs with numeric values only
    Returns scaler, X_train_scaled, X_validate_scaled, X_test_scaled dfs
    """
    scaler = sklearn.preprocessing.StandardScaler().fit(X_train)
    X_train_scaled = pd.DataFrame(scaler.transform(X_train), index = X_train.index, columns = X_train.columns)
    X_validate_scaled = pd.DataFrame(scaler.transform(X_validate), index = X_validate.index, columns = X_validate.columns)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns = X_test.columns)
    return scaler, X_train_scaled, X_validate_scaled, X_test_scaled

In [ ]:
scaler, X_train_scaled, X_validate_scaled, X_test_scaled = Standard_Scaler(X_train, X_validate, X_test)

In [ ]:
explore.head().T

In [ ]:
numeric_cols = ['bathrooms',
 'bedrooms',
 'property_quality',
 'sqft',
 'fips',
 'latitude',
 'longitude',
 'lot_sqft',
 'rawcensustractandblock',
 'regionidcity',
 'zip_code',
 'roomcnt',
 'unitcnt',
 'yearbuilt',
 'structure_value',
 'home_value',
 'land_value',
 'taxamount',
 'home_age']

In [ ]:
binary_vars = []
categorical_vars = ['transactiondate', 'county']

quant_vars = ['LA',
 'Orange',
 'Ventura',
 'fips',
 'latitude',
 'longitude',
 'sqft',
 'lot_sqft',
 'zip_code',
 'property_quality',
 'home_age',
 'logerror']
categorical_target = 'logerror_quartiles'
continuous_target = 'logerror'

In [ ]:
exp.explore_univariate(train, binary_vars, quant_vars)

In [ ]:
df['logerror_quartiles'] = pd.qcut(df.logerror, q=4, labels=['q1', 'q2', 'q3', 'q4'])

In [ ]:
#Standard Scaler
exp.explore_bivariate(train, categorical_target, continuous_target, binary_vars, quant_vars)

### Scatter plot to show cost per sqft and logerror per county 

In [ ]:
plt.rcParams['figure.figsize'] = (12.0, 10.0)
sns.scatterplot(x='home_age', y='logerror',
               data=df, hue='county')
plt.title("How does a homes' age compare to logerror\nwithin each county?")
plt.show()

In [ ]:
df['structure_dollar_per_sqft'] = df.structure_value / df.sqft

In [ ]:
df['land_dollar_per_sqft'] = df.land_value / df.lot_sqft

In [ ]:
df['bed_bath_ratio'] = df.bedrooms / df.bathrooms

In [ ]:
df.head()

In [ ]:
df.structure_dollar_per_sqft.describe()

In [ ]:
plt.rcParams['figure.figsize'] = (15.0, 8.0)
sns.scatterplot(x='structure_dollar_per_sqft', y='logerror',
               data=df, hue='county')
plt.title("How does structure value per sqft compare to logerror\nwithin each county?")
plt.show()

In [ ]:
 df.rename(columns={"bathroomcnt": "bathrooms", 
                   "bedroomcnt": "bedrooms",
                   "buildingqualitytypeid": "property_quality", 
                   "calculatedfinishedsquarefeet": "sqft",
                   "lotsizesquarefeet": "lot_sqft",
                   "regionidzip": "zip_code",
                   "landtaxvaluedollarcnt": "land_value",
                   "structuretaxvaluedollarcnt": "structure_value",
                   "taxvaluedollarcnt ": "home_value"
                  }, inplace=True)
    


In [ ]:
df['logerror_quartiles'] = pd.qcut(df.logerror, q=4, labels=['q1', 'q2', 'q3', 'q4'])

In [ ]:
df.head().T

In [ ]:
# Now we choose what data we want explore by renaming it 

In [ ]:
dummies = pd.get_dummies(df.county) 
dummies.columns = ['LA', 'Orange', 'Ventura']
df = pd.concat([df, dummies], axis=1)

In [ ]:
new = df[['LA', 'Orange', 'Ventura','fips','latitude', 'longitude', 'sqft', 'lot_sqft', 'zip_code', 'property_quality', 'home_age', 'logerror']].copy()